<a href="https://colab.research.google.com/github/Pulsar-kkaturi/DL-Education/blob/master/Notebooks/CNN_FCN_Build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Pulsar-kkaturi/DL-Education.git

# 모듈 불러오기

In [ ]:
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import keras.backend as K
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob

# **경로 설정하기**

In [ ]:
BASE_PATH = r'./DL-Education/dataset/lung_cancer'
IMG_DATA_PATH = os.path.join(BASE_PATH, '2d_images')
MASK_DATA_PATH = os.path.join(BASE_PATH, '2d_masks')

In [ ]:
IMG_DATA_PATH

# **하이퍼 파라미터**
하이퍼 파라미터를 이용해 인공지능 모델을 조정합니다.  
주로 사용하는 하이퍼 파라미터는 다음과 같습니다.

In [ ]:
IMG_HEIGHT, IMG_WIDTH = (96, 96) # 이미지 사이즈
EPOCHS =                       # 훈련 횟수
BATCH_SIZE = 10                   # 한번에 입력할 데이터 개수 
INIT_FILTERS = 32                # 모델 채널 크기
LEARNING_RATE = 2e-4             # 학습률
DECAY_RATE = 0.9                 # 학습률 감소 비율 
ACTIVATION_FN = 'relu'            # 활성함수 ex: relu, elu, sigmoid
DROP_RATE = 0.25                 # 드롭아웃 비율
VAL_RATIO = 0.2                  # 검증 데이터 비율

# **데이터 불러오기**

이미지와 레이블 데이터를 불러옵니다.  
glob.glob 함수를 이용해 경로에 있는 파일들을 검색할 수 있습니다.

In [ ]:
img_path = sorted(glob.glob(os.path.join(IMG_DATA_PATH, '*.tif')))
mask_path = sorted(glob.glob(os.path.join(MASK_DATA_PATH, '*.tif')))

In [ ]:
print(len(img_path), img_path)

In [ ]:
print(len(mask_path), mask_path)

모아놓은 파일들을 다음 함수를 이용해 불러옵니다.

In [ ]:
def load_and_resize_png1(path, img_height, img_width):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.resize(img, dsize=(img_height, img_width), interpolation=cv2.INTER_AREA)
    return img

def load_and_resize_png2(path, img_height, img_width):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.
    img = cv2.resize(img, dsize=(img_height, img_width), interpolation=cv2.INTER_NEAREST)
    return img

def load_and_resize_nii(path, img_height, img_width):
    img = nb.load(path).get_data()
    img = np.transpose(np.squeeze(img), (1, 0))
    img = cv2.resize(img, dsize=(img_height, img_width), interpolation=cv2.INTER_AREA).astype(np.uint8)
    return img

불러온 이미지의 형태는 다음과 같이 (50, 96, 96) 입니다.  
***50*** 장의 세로 ***96***, 가로 ***96*** 의 이미지를 불러왔다는 의미입니다.

In [ ]:
imgs = np.stack([load_and_resize_png1(i_path, IMG_HEIGHT, IMG_WIDTH) for i_path in img_path])
print(imgs.shape)
masks = np.stack([load_and_resize_png2(m_path, IMG_HEIGHT, IMG_WIDTH) for m_path in mask_path])
print(masks.shape)

이미지를 직접 확인해볼까요?  

**왼쪽 이미지**는 우리가 AI 모델에 넣을 **인풋 이미지**고,  
**오른쪽 이미지**는 AI 모델을 학습시킬 때 필요한 정답, 즉 **레이블 이미지**입니다.

레이블 이미지와 같이 우리는 폐 영역을 분리해내는 AI 모델을 만들어 보겠습니다.

In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(10, 20))
for i in range(5):
    ax[i,0].imshow(imgs[i], cmap='gray')
    ax[i,1].imshow(masks[i], cmap='gray')
plt.show()

마스크 데이터도 살펴볼까요?

In [ ]:
masks[4, 40:60, 0:20]

# **데이터 전처리**

AI 모델을 만들기 이전에 모델 학습에 도움을 줄 데이터 전처리를 살펴보겠습니다.  
이 과정을 통해 **AI 모델이 받아들이기 쉬운 형태로 데이터를 가공** 하여 학습 성능을 올릴 수 있습니다.  


먼저, 위에서 본 이미지 데이터를 컴퓨터가 실제로 인식하는 숫자 데이터로 확인해보겠습니다. 

In [ ]:
imgs[0]

범위가 넓고 단위가 큰 숫자들을 정규화하여 학습 성능을 높여봅시다.  
가장 기본적인 정규화 방법으로 고등학교 때 배운 **Z-score** 를 사용할 수 있습니다.

In [ ]:
n_imgs = len(imgs)
imgs = np.reshape(imgs, (n_imgs, -1))
imgs_t = np.transpose(imgs, (1,0))
imgs_t_norm = (imgs_t - np.mean(imgs_t, axis=0)) / np.std(imgs_t, axis=0)
imgs = np.transpose(imgs_t_norm, (1,0))
imgs = np.reshape(imgs, (n_imgs, IMG_HEIGHT, IMG_WIDTH))

다시 값을 확인해 봅시다.

In [ ]:
imgs[0]

그 후 AI 모델에 넣을 수 있는 형태로 이미지 형태를 변형해 보겠습니다. 

In [ ]:
print('변경 전 형태: ', np.shape(imgs))
imgs = imgs[:,:,:,np.newaxis]
masks = masks[:,:,:,np.newaxis]
print('변경 후 형태: ', np.shape(imgs))

마지막으로, 데이터를 학습용 데이터와 검증용 데이터로 나누어 봅시다.  
**학습용 데이터**는 실제로 **모델 학습에 사용** 되는 데이터이고,  
**검증용 데이터**는 학습에 사용하지 않고 학습이 잘 이뤄지는 지 **성능 확인하는 용도**로 사용합니다.

In [ ]:
imgs_train, imgs_val, masks_train, masks_val = train_test_split(imgs, masks, test_size=VAL_RATIO)
print('훈련 데이터 개수: ', len(imgs_train))
print('검증 데이터 개수: ', len(imgs_val))

# **AI 모델을 구성하는 레이어 만들기**

AI 모델은 여러 개의 레이어를 쌓아 올려 만듭니다.  
가장 대표적인 레이어 구조인 **CONV-BN-ACT-POOL** 구조를 만들어 보겠습니다.

먼저 데이터가 들어가는 첫 번째 레이어를 만들어 봅시다.

In [ ]:
layer1 = Input(shape=imgs_train.shape[1:])

그 다음으로 데이터의 특징을 추출할 Convolution 레이어를 연결하겠습니다.

In [ ]:
layer2 = Conv2D(filters=8, kernel_size=(3, 3), activation=None, padding='same')(layer1)

다음으로 레이어 중간에서 정규화를 도와줄 Batch Normalization 레이어를 추가하겠습니다.

In [ ]:
layer3 = BatchNormalization()(layer2)

Batch Normalization 이후 신호를 변환하여 다음 뉴런으로 전달하는 Activation function 레이어를 추가합니다.

In [ ]:
layer4 = Activation('relu')(layer3) #### concat용

이미지 사이즈를 줄여주는 Pooling 레이어를 연결합니다.

In [ ]:
layer5 = MaxPool2D(strides=(2, 2))(layer4)

Conv Block을 한번만 더 반복합시다.

In [ ]:
layer6 = Conv2D(filters=16, kernel_size=(3, 3), activation=None, padding='same')(layer5)

In [ ]:
layer7 = BatchNormalization()(layer6)

In [ ]:
layer8 = Activation('relu')(layer7)

In [ ]:
layer9 = MaxPool2D(strides=(2, 2))(layer8)

다시 Conv-BN-ACT 레이어를 연결합니다.

In [ ]:
layer10 = Conv2D(filters=16, kernel_size=(3, 3), activation=None, padding='same')(layer9)
layer11 = BatchNormalization()(layer10)
layer12 = Activation('relu')(layer11)

이번에는 이미지 사이즈를 키워주는 Upsampling 레이어를 연결합니다.

In [ ]:
layer13 = UpSampling2D(size=(2, 2))(layer12)

Conv block 반복 후 Upsampling 반복

In [ ]:
layer14 = Conv2D(filters=32, kernel_size=(3, 3), activation=None, padding='same')(layer13) 
layer15 = BatchNormalization()(layer14)
layer16 = Activation('relu')(layer15)
layer17 = UpSampling2D(size=(2, 2))(layer16)

마지막으로 아웃풋 레이어를 연결합니다.

In [ ]:
layer18 = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(layer17)

지금까지 만든 레이어를 Model 함수에 넣어 연결하면 모델이 완성됩니다.

In [ ]:
model = Model(layer1, layer18)
model.summary()

# AI 모델 옵션 설정하기

AI 모델을 만든 후 여러가지 훈련 옵션을 설정할 수 있습니다.  
학습률(Learning_rate) 과 손실함수(Loss function), 정확도 기준(Dice_score) 등을 설정해 봅시다.

먼저 정확도 기준으로 Dice score 를 살펴봅시다.  

In [ ]:
 def dice_score(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

In [ ]:
 def dice_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return -(2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

옵션은 complie 을 통해 설정할 수 있습니다.  
모델의 값을 학습시키는 **Optimizer** 는 **Adam** 을 사용하고,  
정답 레이블과 예측값 사이의 차이를 계산하는 **손실 함수**는 **Cross entropy** 를 사용했습니다.   
마지막으로 **정확도 판단 기준**으로는 위에서 만든 **Dice score** 를 사용합니다. 

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=dice_loss, metrics=[dice_score])

# AI 모델 훈련하기

이렇게 준비한 AI 모델을 fit 함수를 이용해 훈련시킬 수 있습니다. 

In [ ]:
history = model.fit(imgs_train, masks_train,
                    batch_size=10,
                    epochs=50,
                    validation_data=(imgs_val, masks_val),
                    shuffle=True)

# 훈련 결과 확인하기

훈련 결과를 한 눈에 확인할 수 있는 그래프를 만들어 보겠습니다.

먼저 그래프의 y축에 사용할 값들을 history 항목에서 추출해 봅시다.

In [ ]:
dice_score = history.history['dice_score']
val_dice_score = history.history['val_dice_score']
loss = history.history['loss']
val_loss = history.history['val_loss']

다음으로 그래프의 x축 값으로 사용할 epoch 을 뽑아 보겠습니다.

In [ ]:
epochs = range(len(dice_score))

각 값들을 확인해볼까요?

In [ ]:
print(loss)
print(val_loss)
print(dice_score)
print(val_dice_score)

이 값들을 이용해 그래프를 만들어 보겠습니다.  

먼저 정확도 그래프를 그려볼까요?

In [ ]:
plt.plot(epochs, dice_score, 'b', color='blue', label='Training score')
plt.plot(epochs, val_dice_score, 'b', color='red', label='Validation score')
plt.title('Training and validation score')
plt.legend()
plt.show()

다음으로 손실 그래프를 살펴보겠습니다.

In [ ]:
plt.plot(epochs, loss, 'b', color='blue', label='Training loss')
plt.plot(epochs, val_loss, 'b', color='red', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# 이미지로 결과 확인하기

마지막으로 AI 모델이 실제로 폐 영역을 잘 분리하는지 이미지로 결과를 확인해 보겠습니다. 

In [ ]:
fix, ax = plt.subplots(5, 3, figsize=(10,20))
for i in range(5):
    pred = model.predict(imgs_train[i][np.newaxis, :, : ,:])
    mask = (pred >= 0.5).astype(np.uint8)
    ax[i, 0].imshow(imgs_train[i, :, :, 0], cmap='gray')
    ax[i, 1].imshow(masks_train[i, :, :, 0], cmap='gray')
    ax[i, 2].imshow(mask[0, :, :, 0], cmap='gray')
plt.show()